In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import sdf_helper as sh

def load_density_data(file_path, den_crit=0.17419597124e28):
    data = sh.getdata(file_path)
    ne = data.Derived_Number_Density_Photon.data / den_crit
    ek = data.Derived_Average_Particle_Energy_Photon.data
    x = data.Grid_Grid_mid.data[0] / 1e-6
    y = data.Grid_Grid_mid.data[1] / 1e-6
    z = data.Grid_Grid_mid.data[2] / 1e-6
    return ne, ek, x, y, z

def energy_stats_in_region(ek, x, y, z, x_range, y_range, z_range):
    x_idx = np.where((x >= x_range[0]) & (x <= x_range[1]))[0]
    y_idx = np.where((y >= y_range[0]) & (y <= y_range[1]))[0]
    z_idx = np.where((z >= z_range[0]) & (z <= z_range[1]))[0]
    if len(x_idx) == 0 or len(y_idx) == 0 or len(z_idx) == 0:
        return None, None, None
    subset = ek[np.ix_(x_idx, y_idx, z_idx)]
    return np.sum(subset), np.var(subset), np.mean(subset)

def ne_stats_in_region(ne, x, y, z, x_range, y_range, z_range):
    x_idx = np.where((x >= x_range[0]) & (x <= x_range[1]))[0]
    y_idx = np.where((y >= y_range[0]) & (y <= y_range[1]))[0]
    z_idx = np.where((z >= z_range[0]) & (z <= z_range[1]))[0]
    if len(x_idx) == 0 or len(y_idx) == 0 or len(z_idx) == 0:
        return None, None, None
    subset = ne[np.ix_(x_idx, y_idx, z_idx)]
    return np.sum(subset), np.mean(subset), np.var(subset)

def energy_density_stats(ne, ek, x, y, z, x_range, y_range, z_range):
    x_idx = np.where((x >= x_range[0]) & (x <= x_range[1]))[0]
    y_idx = np.where((y >= y_range[0]) & (y <= y_range[1]))[0]
    z_idx = np.where((z >= z_range[0]) & (z <= z_range[1]))[0]
    if len(x_idx) == 0 or len(y_idx) == 0 or len(z_idx) == 0:
        return None, None, None
    subset_ne = ne[np.ix_(x_idx, y_idx, z_idx)]
    subset_ek = ek[np.ix_(x_idx, y_idx, z_idx)]
    product = subset_ne * subset_ek
    return np.sum(product), np.mean(product), np.var(product)

def analyze_region_fixed(file_path, x_range, save_csv=False, csv_path=None):
    ne, ek, x, y, z = load_density_data(file_path)
    
    y_range = (-2, 2)
    z_range = (-2, 2)

    ek_total, ek_var, ek_mean = energy_stats_in_region(ek, x, y, z, x_range, y_range, z_range)
    ne_total, ne_mean, ne_var = ne_stats_in_region(ne, x, y, z, x_range, y_range, z_range)
    ed_total, ed_mean, ed_var = energy_density_stats(ne, ek, x, y, z, x_range, y_range, z_range)

    result = {
        "x_range": x_range,
        "ek_total": ek_total,
        "ek_var": ek_var,
        "ek_mean": ek_mean,
        "ne_total": ne_total,
        "ne_mean": ne_mean,
        "ne_var": ne_var,
        "ed_total": ed_total,
        "ed_mean": ed_mean,
        "ed_var": ed_var
    }

    if save_csv and csv_path:
        with open(csv_path, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                x_range[0], x_range[1],
                ek_total, ek_var, ek_mean,
                ne_total, ne_mean, ne_var,
                ed_total, ed_mean, ed_var
            ])
    return result

if __name__ == "__main__":
    file_path = os.path.join(
        '/media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch-set2/epoch3d/ju2024abx25',
        'density0018.sdf'
    )

    output_dir = './output'
    os.makedirs(output_dir, exist_ok=True)

    # x_ranges = [(17.3,18),(18,18.8),(18.8,19.8),(19.8,20.4),
    #             (20.4,21.4),(21.4,22.2),(22.2,22.8),(22.8,23.7),
    #             (23.7,24.8),(24.8,25.8)]
    # x_ranges = [(18.0,18.8),(18.8,19.6),(19.6,20.4),(20.4,21.2),(21.2,22.0),(22.0,23.0),(23.0,23.8),(23.8,24.8),(24.8,25.4)]#35
    # x_ranges = [(16.5, 17.2), (17.2, 17.9), (17.9, 18.7), (18.7, 19.5), (19.5, 20.4),
    #             (20.4, 21.2), (21.2, 22), (22, 22.8), (22.8, 23.5), (23.5, 24.3)]   #0
    x_ranges = [(18, 19),(19, 19.7), (19.7, 20.4), (20.4, 21.2),(21.2,22),(22,22.7),(22.7,23.5)] #25
    csv_path = os.path.join(output_dir, '密度25_固定yz区域统计.csv')

    # 写入表头
    if not os.path.exists(csv_path):
        with open(csv_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                'x_min', 'x_max',
                'ek_total', 'ek_var', 'ek_mean',
                'ne_total', 'ne_mean', 'ne_var',
                'ed_total', 'ed_mean', 'ed_var'
            ])

    for x_min, x_max in x_ranges:
        result = analyze_region_fixed(file_path, (x_min, x_max), save_csv=True, csv_path=csv_path)
        if result:
            print(f"x ∈ ({x_min}, {x_max}) → "
                  f"ne总 = {result['ne_total']:.2e}, "
                  f"ek总 = {result['ek_total']:.2e}, 均值 = {result['ek_mean']:.2e}, 方差 = {result['ek_var']:.2e}, "
                  f"能量密度总 = {result['ed_total']:.2e}")

Reading file /media/lan/4248e56e-6d9a-4026-afad-e8b1d59ceab0/epoch-set2/epoch3d/ju2024abx25/density0018.sdf
t() = time
x(1501, 1001, 1001) = grid
y(1501, 1001, 1001) = grid
z(1501, 1001, 1001) = grid
xc(1500, 1000, 1000) = grid_mid
yc(1500, 1000, 1000) = grid_mid
zc(1500, 1000, 1000) = grid_mid
x ∈ (18, 19) → ne总 = 7.49e+05, ek总 = 5.04e-08, 均值 = 8.68e-15, 方差 = 1.67e-27, 能量密度总 = 1.89e-08
x ∈ (19, 19.7) → ne总 = 3.67e+05, ek总 = 3.23e-08, 均值 = 7.84e-15, 方差 = 3.33e-27, 能量密度总 = 1.20e-08
x ∈ (19.7, 20.4) → ne总 = 3.34e+05, ek总 = 3.17e-08, 均值 = 7.69e-15, 方差 = 2.99e-27, 能量密度总 = 1.17e-08
x ∈ (20.4, 21.2) → ne总 = 4.68e+05, ek总 = 5.04e-08, 均值 = 1.07e-14, 方差 = 5.43e-27, 能量密度总 = 2.07e-08
x ∈ (21.2, 22) → ne总 = 6.98e+05, ek总 = 5.11e-08, 均值 = 1.09e-14, 方差 = 1.75e-27, 能量密度总 = 2.23e-08
x ∈ (22, 22.7) → ne总 = 8.55e+05, ek总 = 5.15e-08, 均值 = 1.27e-14, 方差 = 1.65e-27, 能量密度总 = 2.71e-08
x ∈ (22.7, 23.5) → ne总 = 4.07e+05, ek总 = 3.19e-08, 均值 = 6.79e-15, 方差 = 9.16e-28, 能量密度总 = 1.61e-08
